In [1]:
import numpy as np
import xarray as xr

from tqdm import tqdm
import os



In [2]:
GF_THRESHOLD = 0.0e+0
GF_RANGE     = 1.0e-8
GF_DELTAHBYR = 10156.0e+0
GF_T0        = 360.0e+0
GF_TC        = 905.0e+0

#ATM : Standard atmosphere [Pa]  (Source: NIST, 2014)
ATM     = 1.01325e+5
#AIRMW : Average molecular weight of dry air [g/mol]
AIRMW    = 28.97e+0

In [3]:
folder = '/n/home12/hongwei/HONGWEI/GC_aerosol/rundirs/Volcano/Add_H2SO4/geosfp_4x5_gc_timing_Seb_Sigma0.5/'
file = 'GEOSChem.Restart.NO_H2SO4G.20150101_0000z.nc4'
df = xr.open_dataset(folder+file)

In [4]:
# df

In [5]:
lon = df['lon']
lat = df['lat']
lev = df['lev']

Nx = len(lon)
Ny = len(lat)
Nz = len(lev)

In [6]:
# PCENTER_P: dry pressure

hyam = df['hyam']
hybm = df['hybm']
P0 = df['P0']
P = (hyam + hybm * P0) # [hPa]

P.shape

(47,)

In [7]:
# PCENTER = PCENTER_P + e
# moist air pressure = dry air pressure + vapor pressure
# e: partial water vapor pressure

q = df["Met_SPHU1"] # g/kg
q = q/1000.0 # kg/kg

e = q*0.0

for iz in range(Nz):
    # P: pressure [hPa]
    e[0,iz,:,:] = q[0,iz,:,:]*P[iz] / (0.622 + 0.378*q[0,iz,:,:])
    
e.shape

(1, 47, 91, 144)

In [8]:
# AD: dry air mass [kg] = volume * density = area * dz * density = area * abs(-dp)/g

Area = df['AREA'] # [m2]
dP = df['Met_DELPDRY']*100.0 # [Pa]
g = 9.8 # [m/s2]

AD = dP*0.0
for iz in range(Nz):
    AD[0,iz,:,:] = Area[:,:] * dP[0,iz,:,:] / g
    
AD.shape

(1, 47, 91, 144)

In [9]:
# TCENTER: temperature
T = df['Met_TMPU1'] # [K]

T.shape

(1, 47, 91, 144)

In [10]:
# Spc_SO4: 

Spc_SO4 = df['SpeciesRst_SO4'] # mol mol-1

Spc_SO4.shape

(1, 47, 91, 144)

## InTroposphere is set to be 0 now, which need to be update in the future!!!

In [11]:
SO4_MW_G = 96.0 # g/mol

# Calculate H2SO4 gas phase prefactors
GF_INVT0 = 1.0/GF_T0
GF_LOGP0 = -1.0*GF_DELTAHBYR*GF_INVT0 + 16.259
GF_BFACTOR = 0.38/(GF_TC - GF_T0)
GF_ATMCONV = np.log(ATM)


InTroposphere = False

TCENTER = T
PCENTER_P = e*0.0
# P
PCENTER = e*0.0
# PCENTER_P + e
        
for iz in range(Nz):
    PCENTER_P[0,iz,:,:] = P[iz]
    PCENTER[0,iz,:,:] = PCENTER_P[0,iz,:,:] + e[0,iz,:,:]
    
PCENTER.shape, PCENTER_P.shape, e.shape, P.shape

((1, 47, 91, 144), (1, 47, 91, 144), (1, 47, 91, 144), (47,))

In [12]:
# calcualte the gas-liquid partition for H2SO4:
    # TCENTER = State_Met%T(I,J,L).
    # PCENtER = State_Met%PMID(I,J,L): moist air pressure.
    # PCENTER_P = State_Met%PMID_DRY(I,J,L): Dry air partial pressure for part. P calc.
    # AD = State_Met%AD(I,J,L), ! Dry air mass [kg] in grid box
    # InTroposphere = State_Met%InTroposphere(I,J,L)
    # Spc_SO4 = Spc(I,J,L,id_SO4), [mol/mol]
    
    # SO4_MW_G = State_Chm%SpcData(id_SO4)%Info%emMW_g ! g/mol
    

INVAIR = AIRMW / AD
print(INVAIR.shape)

H2SO4SUM = Spc_SO4 * INVAIR / SO4_MW_G
print(H2SO4SUM.shape)

GF_PP = H2SO4SUM * PCENTER_P
print(GF_PP.shape)

GF_INVT = 1.0/TCENTER
print(GF_INVT.shape)

GF_CFACTOR = 1.0 + np.log(GF_T0*GF_INVT) - GF_T0*GF_INVT
print(GF_CFACTOR.shape)

GF_LOGPSULFATE = GF_LOGP0 + GF_DELTAHBYR * (GF_INVT0 - GF_INVT + GF_BFACTOR*GF_CFACTOR)
print(GF_LOGPSULFATE.shape)

GF_LOGPSULFATE = GF_LOGPSULFATE + GF_ATMCONV
print(GF_LOGPSULFATE.shape)

GF_PVAP = 1.0e-2 * np.exp(GF_LOGPSULFATE)
print(GF_PVAP.shape)

GF_DIFF = (GF_PVAP+GF_THRESHOLD) - GF_PP
print(GF_DIFF.shape)


AERFRAC = GF_DIFF*0.0 + 1.0

for ix in tqdm(range(Nx)):
    for iy in range(Ny):
        for iz in range(Nz):
            
            if PCENTER[0,iz,iy,ix]>=100.0:
                AERFRAC[0,iz,iy,ix] = 1.0
                
            elif InTroposphere:
                AERFRAC[0,iz,iy,ix] = 1.0
                
            else:
                if GF_DIFF[0,iz,iy,ix] < 0.0:
                    AERFRAC[0,iz,iy,ix] = 1.0
                elif GF_DIFF[0,iz,iy,ix] < GF_RANGE:
                    AERFRAC[0,iz,iy,ix] = 1.0 - (GF_DIFF[0,iz,iy,ix]/GF_RANGE)
                else:
                    AERFRAC[0,iz,iy,ix] = 0.0

  0%|          | 0/144 [00:00<?, ?it/s]

(1, 47, 91, 144)
(1, 47, 91, 144)
(1, 47, 91, 144)
(1, 47, 91, 144)
(1, 47, 91, 144)
(1, 47, 91, 144)
(1, 47, 91, 144)
(1, 47, 91, 144)
(1, 47, 91, 144)


100%|██████████| 144/144 [10:48<00:00,  4.51s/it]


In [13]:
def func(TCENTER, PCENTER, PCENTER_P, AD, InTroposphere, Spc_SO4):
    # TCENTER = State_Met%T(I,J,L).
    # PCENtER = State_Met%PMID(I,J,L): moist air pressure.
    # PCENTER_P = State_Met%PMID_DRY(I,J,L): Dry air partial pressure for part. P calc.
    # AD = State_Met%AD(I,J,L), ! Dry air mass [kg] in grid box
    # InTroposphere = State_Met%InTroposphere(I,J,L)
    # Spc_SO4 = Spc(I,J,L,id_SO4), [mol/mol]
    
    # SO4_MW_G = State_Chm%SpcData(id_SO4)%Info%emMW_g ! g/mol
    
    INVAIR = AIRMW / AD
            
    if PCENTER>=100.0:
        AERFRAC = 1.0
    elif InTroposphere:
        AERFRAC = 1.0
    else:
        H2SO4SUM = Spc_SO4 * INVAIR / SO4_MW_G
        
        GF_PP = H2SO4SUM * PCENTER_P
        GF_INVT = 1.0/TCENTER
        GF_CFACTOR = 1.0 + np.log(GF_T0*GF_INVT) - GF_T0*GF_INVT
        
        GF_LOGPSULFATE = GF_LOGP0 + GF_DELTAHBYR * (GF_INVT0 - GF_INVT + GF_BFACTOR*GF_CFACTOR)
        
        GF_LOGPSULFATE = GF_LOGPSULFATE + GF_ATMCONV
        
        GF_PVAP = 1.0e-2 * np.exp(GF_LOGPSULFATE)
        GF_DIFF = (GF_PVAP+GF_THRESHOLD) - GF_PP
        
        if GF_DIFF < 0.0:
            AERFRAC = 1.0
        elif GF_DIFF < GF_RANGE:
            AERFRAC = 1.0 - (GF_DIFF/GF_RANGE)
        else:
            AERFRAC = 0.0

In [14]:
df

<xarray.Dataset>
Dimensions:              (ilev: 48, lat: 91, lev: 47, lon: 144, time: 1)
Coordinates:
  * time                 (time) datetime64[ns] 2016-07-01T00:20:00
  * lev                  (lev) float64 0.9925 0.9775 ... 0.0001387 3.8e-05
  * ilev                 (ilev) float64 1.0 0.985 0.97 ... 6.6e-05 1e-05
  * lat                  (lat) float64 -89.5 -88.0 -86.0 ... 86.0 88.0 89.5
  * lon                  (lon) float64 -180.0 -177.5 -175.0 ... 175.0 177.5
Data variables:
    hyam                 (lev) float64 ...
    hybm                 (lev) float64 ...
    hyai                 (ilev) float64 ...
    hybi                 (ilev) float64 ...
    P0                   float64 ...
    AREA                 (lat, lon) float32 ...
    Met_TMPU1            (time, lev, lat, lon) float32 215.0 215.0 ... 195.5
    Met_SPHU1            (time, lev, lat, lon) float32 0.0134 ... 0.004332
    Met_PS1DRY           (time, lat, lon) float32 ...
    Met_PS1WET           (time, lat, lon) float32 ...
    Met_DELPDRY          (time, lev, lat, lon) float32 10.1 10.1 ... 0.056 0.056
    Chem_WetDepNitrogen  (time, lat, lon) float32 ...
    Chem_DryDepNitrogen  (time, lat, lon) float32 ...
    Chem_SO2AfterChem    (time, lev, lat, lon) float32 ...
    Chem_H2O2AfterChem   (time, lev, lat, lon) float32 ...
    SpeciesRst_SOAS      (time, lev, lat, lon) float32 ...
    SpeciesRst_SOAP      (time, lev, lat, lon) float32 ...
    SpeciesRst_SO4s      (time, lev, lat, lon) float32 ...
    SpeciesRst_SO4       (time, lev, lat, lon) float32 ...
    SpeciesRst_SO2       (time, lev, lat, lon) float32 ...
    SpeciesRst_SALC      (time, lev, lat, lon) float32 ...
    SpeciesRst_SALA      (time, lev, lat, lon) float32 ...
    SpeciesRst_pFe       (time, lev, lat, lon) float32 ...
    SpeciesRst_OCPO      (time, lev, lat, lon) float32 ...
    SpeciesRst_OCPI      (time, lev, lat, lon) float32 ...
    SpeciesRst_NITs      (time, lev, lat, lon) float32 ...
    SpeciesRst_NIT       (time, lev, lat, lon) float32 ...
    SpeciesRst_NH4       (time, lev, lat, lon) float32 ...
    SpeciesRst_NH3       (time, lev, lat, lon) float32 ...
    SpeciesRst_MSA       (time, lev, lat, lon) float32 ...
    SpeciesRst_H2O2      (time, lev, lat, lon) float32 ...
    SpeciesRst_DST4      (time, lev, lat, lon) float32 ...
    SpeciesRst_DST3      (time, lev, lat, lon) float32 ...
    SpeciesRst_DST2      (time, lev, lat, lon) float32 ...
    SpeciesRst_DST1      (time, lev, lat, lon) float32 ...
    SpeciesRst_DMS       (time, lev, lat, lon) float32 ...
    SpeciesRst_BCPO      (time, lev, lat, lon) float32 ...
    SpeciesRst_BCPI      (time, lev, lat, lon) float32 ...
Attributes:
    title:                           GEOS-Chem diagnostic collection: Restart
    history:                         
    format:                          NetCDF-3 with large file support
    conventions:                     COARDS
    ProdDateTime:                    
    reference:                       www.geos-chem.org; wiki.geos-chem.org
    contact:                         GEOS-Chem Support Team (geos-chem-suppor...
    simulation_start_date_and_time:  2016-07-01 00:00:00z
    simulation_end_date_and_time:    2016-07-01 00:20:00z

## Partition H2SO4 into Gas and Liquid phases

In [15]:
AERFRAC.shape, Spc_SO4.shape

Spc_H2SO4G = Spc_SO4*0.0
Spc_H2SO4G = Spc_SO4 * (1-AERFRAC)
Spc_SO4_new = Spc_SO4 * AERFRAC

In [16]:
Spc_H2SO4G.attrs['long_name'] = 'Dry mixing ratio of species H2SO4G'
Spc_H2SO4G.attrs['units'] = 'mol mol-1 dry'
Spc_H2SO4G.attrs['averaging_method'] = 'instantaneous'

Spc_SO4_new.attrs['long_name'] = 'Dry mixing ratio of species SO4'
Spc_SO4_new.attrs['units'] = 'mol mol-1 dry'
Spc_SO4_new.attrs['averaging_method'] = 'instantaneous'

Spc_H2SO4G

<xarray.DataArray (time: 1, lev: 47, lat: 91, lon: 144)>
array([[[[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         ...,
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

        [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
          0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
...
         [8.2838866e-12, 8.3181908e-12, 8.3569185e-12, ...,
          8.2091322e-12, 8.2279227e-12, 8.2550113e-12],
         [8.6244839e-12, 8.6324680e-12, 8.6411000e-12, ...,
          8.6142751e-12, 8.6150288e-12, 8.6182016e-12],
         [9.6405939e-12, 9.6405939e-12, 9.6405939e-12, ...,
          9.6405939e-12, 9.6405939e-12, 9.6405939e-12]],

        [[4.2591868e-11, 4.2591868e-11, 4.2591868e-11, ...,
          4.2591868e-11, 4.2591868e-11, 4.2591868e-11],
         [2.4164655e-11, 2.3924251e-11, 2.3718742e-11, ...,
          2.5030544e-11, 2.4718080e-11, 2.4425727e-11],
         [1.5071612e-11, 1.4782160e-11, 1.4536227e-11, ...,
          1.6417177e-11, 1.5901029e-11, 1.5430161e-11],
         ...,
         [2.2601872e-16, 2.2600549e-16, 2.2599337e-16, ...,
          2.2852274e-16, 2.2604879e-16, 2.2603318e-16],
         [2.3645851e-16, 2.3645851e-16, 2.3645851e-16, ...,
          2.3645851e-16, 2.3645851e-16, 2.3645851e-16],
         [2.6108962e-16, 2.6108962e-16, 2.6108962e-16, ...,
          2.6108962e-16, 2.6108962e-16, 2.6108962e-16]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2016-07-01T00:20:00
  * lev      (lev) float64 0.9925 0.9775 0.9625 ... 0.0004141 0.0001387 3.8e-05
  * lat      (lat) float64 -89.5 -88.0 -86.0 -84.0 -82.0 ... 84.0 86.0 88.0 89.5
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
Attributes:
    long_name:         Dry mixing ratio of species H2SO4G
    units:             mol mol-1 dry
    averaging_method:  instantaneous

In [17]:
df2 = df
df2['SpeciesRst_H2SO4G'] = Spc_H2SO4G
df2['SpeciesRst_SO4'] = Spc_SO4_new


df2.to_netcdf("GEOSChem.Restart.H2SO4G.20150101_0000z.nc4")

In [18]:
df2

<xarray.Dataset>
Dimensions:              (ilev: 48, lat: 91, lev: 47, lon: 144, time: 1)
Coordinates:
  * time                 (time) datetime64[ns] 2016-07-01T00:20:00
  * lev                  (lev) float64 0.9925 0.9775 ... 0.0001387 3.8e-05
  * ilev                 (ilev) float64 1.0 0.985 0.97 ... 6.6e-05 1e-05
  * lat                  (lat) float64 -89.5 -88.0 -86.0 ... 86.0 88.0 89.5
  * lon                  (lon) float64 -180.0 -177.5 -175.0 ... 175.0 177.5
Data variables:
    hyam                 (lev) float64 0.02402 3.321 9.864 ... 0.1387 0.038
    hybm                 (lev) float64 0.9925 0.9742 0.9526 ... 0.0 0.0 0.0
    hyai                 (ilev) float64 0.0 0.04805 6.594 ... 0.2113 0.066 0.01
    hybi                 (ilev) float64 1.0 0.985 0.9634 0.9419 ... 0.0 0.0 0.0
    P0                   float64 1e+03
    AREA                 (lat, lon) float32 2.701e+08 2.701e+08 ... 2.701e+08
    Met_TMPU1            (time, lev, lat, lon) float32 215.0 215.0 ... 195.5
    Met_SPHU1            (time, lev, lat, lon) float32 0.0134 ... 0.004332
    Met_PS1DRY           (time, lat, lon) float32 674.3 674.3 ... 1.006e+03
    Met_PS1WET           (time, lat, lon) float32 674.3 674.3 ... 1.007e+03
    Met_DELPDRY          (time, lev, lat, lon) float32 10.1 10.1 ... 0.056 0.056
    Chem_WetDepNitrogen  (time, lat, lon) float32 1.11e-16 ... 4.574e-17
    Chem_DryDepNitrogen  (time, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    Chem_SO2AfterChem    (time, lev, lat, lon) float32 ...
    Chem_H2O2AfterChem   (time, lev, lat, lon) float32 ...
    SpeciesRst_SOAS      (time, lev, lat, lon) float32 ...
    SpeciesRst_SOAP      (time, lev, lat, lon) float32 ...
    SpeciesRst_SO4s      (time, lev, lat, lon) float32 ...
    SpeciesRst_SO4       (time, lev, lat, lon) float32 8.534e-12 ... 4.132e-11
    SpeciesRst_SO2       (time, lev, lat, lon) float32 ...
    SpeciesRst_SALC      (time, lev, lat, lon) float32 ...
    SpeciesRst_SALA      (time, lev, lat, lon) float32 ...
    SpeciesRst_pFe       (time, lev, lat, lon) float32 ...
    SpeciesRst_OCPO      (time, lev, lat, lon) float32 ...
    SpeciesRst_OCPI      (time, lev, lat, lon) float32 ...
    SpeciesRst_NITs      (time, lev, lat, lon) float32 ...
    SpeciesRst_NIT       (time, lev, lat, lon) float32 ...
    SpeciesRst_NH4       (time, lev, lat, lon) float32 ...
    SpeciesRst_NH3       (time, lev, lat, lon) float32 ...
    SpeciesRst_MSA       (time, lev, lat, lon) float32 ...
    SpeciesRst_H2O2      (time, lev, lat, lon) float32 ...
    SpeciesRst_DST4      (time, lev, lat, lon) float32 ...
    SpeciesRst_DST3      (time, lev, lat, lon) float32 ...
    SpeciesRst_DST2      (time, lev, lat, lon) float32 ...
    SpeciesRst_DST1      (time, lev, lat, lon) float32 ...
    SpeciesRst_DMS       (time, lev, lat, lon) float32 ...
    SpeciesRst_BCPO      (time, lev, lat, lon) float32 ...
    SpeciesRst_BCPI      (time, lev, lat, lon) float32 ...
    SpeciesRst_H2SO4G    (time, lev, lat, lon) float32 0.0 0.0 ... 2.611e-16
Attributes:
    title:                           GEOS-Chem diagnostic collection: Restart
    history:                         
    format:                          NetCDF-3 with large file support
    conventions:                     COARDS
    ProdDateTime:                    
    reference:                       www.geos-chem.org; wiki.geos-chem.org
    contact:                         GEOS-Chem Support Team (geos-chem-suppor...
    simulation_start_date_and_time:  2016-07-01 00:00:00z
    simulation_end_date_and_time:    2016-07-01 00:20:00z